In [2]:
import pickle

In [3]:
import glob
import errno
import unicodedata

In [3]:
testing = "1928_in_association_football 0 The following are the football -LRB- soccer -RRB- events of the year 1928 throughout the world ."
testing1 = testing.split()
print(testing1)
testing2 = " ".join(testing1[2:])
print(type(testing2))

['1928_in_association_football', '0', 'The', 'following', 'are', 'the', 'football', '-LRB-', 'soccer', '-RRB-', 'events', 'of', 'the', 'year', '1928', 'throughout', 'the', 'world', '.']
<class 'str'>


In [4]:
path = 'wiki-pages-text/*'
files = glob.glob(path)
#print(files[0])
#name = files[0]
count = 0
faultySentenceIdCount = 0
wikiShards = {}

for name in files:
    count += 1
    print(count)
    with open(name, encoding="utf8") as f:
        for line in f:
            eachLine = line.split()
            docId = unicodedata.normalize('NFD', eachLine[0])
            tempSentenceId = eachLine[1]
            if tempSentenceId.isdigit():
                
                sentenceId = int(tempSentenceId)
                sentence = " ".join(eachLine[2:])
                newSentence = {sentenceId:sentence}
                
                if docId in wikiShards:
                    
                    wikiShards[docId].append(newSentence)
                    
                else:
                    
                    wikiShards[docId] = []
                    wikiShards[docId].append(newSentence)
            else:
                faultySentenceIdCount += 1


with open('allWikiShardsDict.pickle', 'wb') as handle:
    pickle.dump(wikiShards, handle, protocol=pickle.HIGHEST_PROTOCOL)            
print(count)
print(faultySentenceIdCount)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
109
499


In [5]:
with open('allWikiShardsDict.pickle', 'rb') as handle:
    allWikiDocs = pickle.load(handle)

# print (wikiShards == allWikiDocs)
print("done")

done


In [6]:
listKeys = list(allWikiDocs.keys())

docId_int_dict = {}

i = 0
for key in listKeys:
    docId_int_dict[key] = i
    i += 1

with open('docIdToIntDict.pickle', 'wb') as handle:
    pickle.dump(docId_int_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    


In [7]:
with open('docIdToIntDict.pickle', 'rb') as handle:
    docIdIntDict = pickle.load(handle)
    
#print(docId_int_dict == docIdIntDict)

In [8]:
import json

with open('train.json', 'r') as f:
    trainJsonDict = json.load(f)
    
print(len(trainJsonDict.keys()))

145449


In [9]:
trainingWikiDocs = {}
for key in trainJsonDict.keys():
    tempEvidenceList = trainJsonDict[key]['evidence']  
    for evidence in tempEvidenceList:
        foundDoc = unicodedata.normalize('NFD', evidence[0])
        if (foundDoc in allWikiDocs) and (foundDoc not in trainingWikiDocs):
            trainingWikiDocs[foundDoc] = allWikiDocs[foundDoc]
            
print("done")

done


In [10]:
with open('allTrainShards.pickle', 'wb') as handle:
    pickle.dump(trainingWikiDocs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
with open('allTrainShards.pickle', 'rb') as handle:
    allTrainDocs = pickle.load(handle)
    
print(allTrainDocs == trainingWikiDocs)

True


In [21]:
print(allTrainDocs["Papua_New_Guinea"])

[{0: 'Papua New Guinea -LRB- PNG ; -LSB- ˈpæpuə_njuː_ˈgɪniː , _ ˈpɑː - , _ - pju - -RSB- , -LSB- ˈpæpjuə , _ pɑːˈpuːə -RSB- Papua Niugini Hiri Motu : Papua Niu Gini -RRB- , officially the Independent State of Papua New Guinea , is an Oceanian country that occupies the eastern half of the island of New Guinea and its offshore islands in Melanesia , a region of the southwestern Pacific Ocean north of Australia .'}, {1: 'Its capital , located along its southeastern coast , is Port Moresby .'}, {2: 'The western half of New Guinea forms the Indonesian provinces of Papua and West Papua .'}, {5: 'At the national level , after being ruled by three external powers since 1884 , Papua New Guinea established its sovereignty in 1975 .'}, {6: 'This followed nearly 60 years of Australian administration , which started during World War I .'}, {7: 'It became an independent Commonwealth realm in 1975 with Queen Elizabeth II as its head of state and became a member of the Commonwealth of Nations in its o